In [14]:
using VideoIO
using Plots
using Images
using LaTeXStrings
using Measures
include("funciones.jl")

difusión

In [2]:
function aplica_kernel(color,kernel,n, x,y)
    vecinos = [color[x+i, y+j] for i in -n:n, j in -n:n] #establece la vecindad alrededor del pixel
    suma = sum(vecinos .* kernel) #multiplica la vecidad por el kernel entrada por entrada y suma todos los valores
    
    #si la suma es mayor a 1, devuelve 1; si la suma es menor a 0, devuelve 0; el cualquier otro caso devuelve el valor calculado
    if suma<0 suma=0
    elseif suma>1 suma=1
    else suma
    end
    
    return suma
end
function convolucion(Imagen, kernel)
    tamaño = size(Imagen)
    n = Int((size(kernel)[1]-1)/2)
    
    #crea bordes de tamaño n alrededor de la imagen y al mismo tiempo descompone en rojo, verde y azul
    RED = zeros(tamaño[2]+2n, tamaño[1]+2n)
    GREEN = zeros(tamaño[2]+2n, tamaño[1]+2n)
    BLUE = zeros(tamaño[2]+2n, tamaño[1]+2n)
    for i in 1:tamaño[2]+(2*n)
        for j in 1:tamaño[1]+(2*n)
            if (i<=n || i>tamaño[2]+n || j<=n || j>tamaño[1]+n)
                RED[i,j] = 1
                GREEN[i,j] = 1
                BLUE[i,j] = 1
            else
                RED[i,j] = red(Imagen[j-n,i-n])
                GREEN[i,j] = green(Imagen[j-n,i-n])
                BLUE[i,j] = blue(Imagen[j-n,i-n])
            end
        end
    end
    
    #almacena los nuevos valores de rojo, verde y azul ya aplicada la convolucion
    RED_nueva = [aplica_kernel(RED,kernel,n, i,j) for i in n+1:size(RED)[1]-n, j in n+1:size(RED)[2]-n]
    GREEN_nueva = [aplica_kernel(GREEN,kernel,n, i,j) for i in n+1:size(GREEN)[1]-n, j in n+1:size(GREEN)[2]-n]
    BLUE_nueva = [aplica_kernel(BLUE,kernel,n, i,j) for i in n+1:size(BLUE)[1]-n, j in n+1:size(BLUE)[2]-n]
    #crea la nueva imagen con la composición de rojo, verde y azul
    nueva_imagen = [RGB(RED_nueva[i,j], GREEN_nueva[i,j], BLUE_nueva[i,j]) for j in n+1:size(RED_nueva)[2]-n, i in n+1:size(RED_nueva)[1]-n]
    
    return nueva_imagen
end

convolucion (generic function with 1 method)

In [18]:
video = VideoIO.load("videoparticulas.mp4");
tam_x, tam_y = size(video[1])[1], size(video[1])[2]
frontera = [[CartesianIndex(x, y) for x in [1:10; tam_x-9:tam_x] for y in 1:tam_y];
            [CartesianIndex(x, y) for x in 10:tam_x-9 for y in [1:10; tam_y-9:tam_y]]];

In [13]:
kernel1 = [1 1 1; 1 -8 1; 1 1 1]
img1 = Gray.(convolucion(video[1], kernel1))
save("laplacian+1-8.png", img1)

kernel11 = [-1 -1 -1; -1 8 -1; -1 -1 -1]
img11 = Gray.(convolucion(video[1], kernel11))
save("laplacian-1+8.png", img11)

kernel2 = [
0 1 1 2 2 2 1 1 0;
1 2 4 5 5 5 4 2 1;
1 4 5 3 0 3 5 4 1;
2 5 3 -12 -24 -12 3 5 2;
2 5 0 -24 -40 -24 0 5 2;
2 5 3 -12 -24 -12 3 5 2;
1 4 5 3 0 3 5 4 1;
1 2 4 5 5 5 4 2 1;
0 1 1 2 2 2 1 1 0
    ]
img2 = Gray.(convolucion(video[1], kernel2))
save("log01122.png", img2)

kernel3 = [
0 0 3 2 2 2 3 0 0;
0 2 3 5 5 5 3 2 0;
3 3 5 3 0 3 5 3 3;
2 5 3 -12 -23 -12 3 2 2;
2 5 0 -23 -40 -23 0 5 2;
2 5 3 -12 -23 -12 3 2 2;
3 3 5 3 0 3 5 3 3;
0 2 3 5 5 5 3 2 0;
0 0 3 2 2 2 3 0 0
    ]
img3 = Gray.(convolucion(video[1], kernel3))
save("log00322.png", img3)

kernel4 = [
0 0 -1 0 0;
0 -1 -2 -1 0;
-1 -2 16 -2 -1;
0 -1 -2 -1 0;
0 0 -1 0 0
    ]
img4 = Gray.(convolucion(video[1], kernel4))
save("log00-1.png", img4)

In [21]:
#Y la más importante, la buena!
#Son dos imágenes, una binaria y una que tiene los centroides
save("binaria.png", procesar_imagen!(video[1]))
A = zeros(tam_x, tam_y)
A[encontrar_partículas_1_frame!(video[1])] .= 1
save("centroides.png", Gray.(A))